In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

Load Data

In [2]:
seed = 0
rng = np.random.RandomState(seed)
np.seterr(all='raise')

dataset = 'mnist_784'   # full size 70000
data_size = 50000  # Training + validation data size
data_size_test = 10000  # Test data size

#dataset = 'Fashion-MNIST'   # full size 70000
#data_size = 60000  # Training + validation data size
#data_size_test = 10000  # Test data size

#dataset = 'EMNIST_Balanced' # full size 131600
#data_size = 120000  # Training + validation data size
#data_size_test = 10000  # Test data size

data = fetch_openml(dataset)
X_full = data.data
y_full = np.float32(data.target)

num_images = X_full.shape[0]
in_size = X_full.shape[1]

if dataset == 'EMNIST_Balanced':
  num_output = 47
else:
  num_output = 10

rand_perm = rng.permutation(num_images)
    
X_full = (X_full - X_full.min()) / ((X_full.max()+0.00001) - X_full.min())
X_full = X_full.to_numpy()
    
X_full = X_full[rand_perm]
y_full = y_full[rand_perm]

X_train = X_full[:data_size]
y_train_data = y_full[:data_size]

X_test = X_full[-data_size_test:]
y_test_data = y_full[-data_size_test:]

In [3]:
y_train = np.zeros((data_size, num_output))
for i in range(y_train.shape[0]):
  y_train[i,np.int32(y_train_data[i])] = 1;

y_test = np.zeros((data_size_test, num_output))
for i in range(y_test_data.shape[0]):
  y_test[i,np.int32(y_test_data[i])] = 1;

Fill spike dataset

In [4]:
num_iter = 250
max_in_spikes = 200
batch_size = 200

'''
num_iter = 250
max_in_spikes = 200
batch_size = 200

X_in_train = np.zeros((X_train.shape[0], num_iter, X_train.shape[1]), dtype=np.int8)
X_in_test = np.zeros((X_test.shape[0], num_iter, X_test.shape[1]), dtype=np.int8)

num_batches = data_size//batch_size
num_batches_test = data_size_test//batch_size
for b in range(num_batches):
  if(b%50==0):
    print("completed: ", b/num_batches)
  batch_data = X_train[b*batch_size:(b+1)*batch_size]*(max_in_spikes/num_iter)
  batch_reshp = np.reshape(batch_data, (batch_size, -1, 1))
  batch_data_tiled = np.tile(batch_reshp, (1, 1, num_iter))
  batch_data_trans = np.transpose(batch_data_tiled, (0,2,1))
  batch_in = np.random.uniform(size=(batch_size , num_iter, in_size))
  batch_in[batch_in<batch_data_trans] = 1
  batch_in[batch_in<1] = 0
  X_in_train[b*batch_size:(b+1)*batch_size, :, :] = batch_in

for b in range(num_batches_test):
  batch_data = X_test[b*batch_size:(b+1)*batch_size]*(max_in_spikes/num_iter)
  batch_reshp = np.reshape(batch_data, (batch_size, -1, 1))
  batch_data_tiled = np.tile(batch_reshp, (1, 1, num_iter))
  batch_data_trans = np.transpose(batch_data_tiled, (0,2,1))
  batch_in = np.random.uniform(size=(batch_size , num_iter, in_size))
  batch_in[batch_in<batch_data_trans] = 1
  batch_in[batch_in<1] = 0
  X_in_test[b*batch_size:(b+1)*batch_size, :, :] = batch_in
'''
# can't load dataset in low RAM, need to generate on the fly

'\nnum_iter = 250\nmax_in_spikes = 200\nbatch_size = 200\n\nX_in_train = np.zeros((X_train.shape[0], num_iter, X_train.shape[1]), dtype=np.int8)\nX_in_test = np.zeros((X_test.shape[0], num_iter, X_test.shape[1]), dtype=np.int8)\n\nnum_batches = data_size//batch_size\nnum_batches_test = data_size_test//batch_size\nfor b in range(num_batches):\n  if(b%50==0):\n    print("completed: ", b/num_batches)\n  batch_data = X_train[b*batch_size:(b+1)*batch_size]*(max_in_spikes/num_iter)\n  batch_reshp = np.reshape(batch_data, (batch_size, -1, 1))\n  batch_data_tiled = np.tile(batch_reshp, (1, 1, num_iter))\n  batch_data_trans = np.transpose(batch_data_tiled, (0,2,1))\n  batch_in = np.random.uniform(size=(batch_size , num_iter, in_size))\n  batch_in[batch_in<batch_data_trans] = 1\n  batch_in[batch_in<1] = 0\n  X_in_train[b*batch_size:(b+1)*batch_size, :, :] = batch_in\n\nfor b in range(num_batches_test):\n  batch_data = X_test[b*batch_size:(b+1)*batch_size]*(max_in_spikes/num_iter)\n  batch_reshp 

In [5]:
#tf tiling practice
arr = np.array([[1,2,3,4],[5,6,7,8]])
a = tf.constant(arr)
c = tf.reshape(a, [2,-1,1])
d = tf.tile(c, [1,1,4])
print(a)
print(d[0,:,:])
print(d[1,:,:])

tf.Tensor(
[[1 2 3 4]
 [5 6 7 8]], shape=(2, 4), dtype=int64)
tf.Tensor(
[[1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]
 [4 4 4 4]], shape=(4, 4), dtype=int64)
tf.Tensor(
[[5 5 5 5]
 [6 6 6 6]
 [7 7 7 7]
 [8 8 8 8]], shape=(4, 4), dtype=int64)


Define Weight matrix

In [6]:
Nx = 12
Ny = 12
Nz = 12
N = Nx*Ny*Nz
LqW = 3
in_conn_density = 0.15
inh_fr = 0.2
lam = 9

W_lsm = np.zeros((N,N))
W_in = np.zeros((in_size,N))

in_conn_range = np.int32(N*in_conn_density)
for i in range(in_size):
  input_perm_i = np.arange(N)
  np.random.shuffle(input_perm_i)
  pos_conn = input_perm_i[:in_conn_range]
  neg_conn = input_perm_i[-in_conn_range:]
  W_in[i,pos_conn] = LqW
  W_in[i,neg_conn] = -LqW

input_perm = np.arange(N)
np.random.shuffle(input_perm) # first 0.2*N indices are inhibitory
inh_range = np.int32(inh_fr*N) # indices 0 to inh_range-1 are inhibitory

for i in range(N):
  posti = input_perm[i] # input_perm[i] is the post-neuron index
  zi = posti//(Nx*Ny)
  yi = (posti-zi*Nx*Ny)//Nx
  xi = (posti-zi*Nx*Ny)%Nx
  for j in range(N):
    prej = input_perm[j] # input_perm[j] is the pre-neuron index
    zj = prej//(Nx*Ny)
    yj = (prej-zj*Nx*Ny)//Nx
    xj = (prej-zj*Nx*Ny)%Nx
    D = ((xi-xj)**2 + (yi-yj)**2 + (zi-zj)**2)
    if i<inh_range and j<inh_range: # II connection, C = 0.3
      P = 0.3*np.exp(-D/lam)
      Pu1 = np.random.uniform()
      if Pu1<P:
        W_lsm[prej,posti] = -LqW
    if i<inh_range and j>=inh_range: # EI connection, C = 0.1
      P = 0.1*np.exp(-D/lam)
      Pu1 = np.random.uniform()
      if Pu1<P:
        W_lsm[prej,posti] = LqW
    if i>=inh_range and j<inh_range: # IE connection, C = 0.05
      P = 0.05*np.exp(-D/lam)
      Pu1 = np.random.uniform()
      if Pu1<P:
        W_lsm[prej,posti] = -LqW
    if i>=inh_range and j>=inh_range: # EE connection, C = 0.2
      P = 0.2*np.exp(-D/lam)
      Pu1 = np.random.uniform()
      if Pu1<P:
        W_lsm[prej,posti] = LqW

for i in range(N):
  W_lsm[i,i] = 0

print("average fan out: ", np.mean(np.sum(W_lsm, axis=1)/LqW))

average fan out:  11.87962962962963


Define Network

In [7]:

def run_LSM(Vm, SI, Sin, SinI, eta, eta1, a1, a2, in_W, W, sc_W, th, n_iter):
  Sliq = []
  for i in range(n_iter):
    Vm = (1-eta)*Vm + a1*(tf.matmul(SI, sc_W*W) + tf.matmul(SinI, in_W))
    Sout = tf.cast(Vm>=th, dtype=tf.float32)
    Vm = Vm*tf.cast(Vm<th, dtype=tf.float32)
    SinI = (1-eta1)*SinI + a2*Sin[:,i,:]
    SI = (1-eta1)*SI + a2*Sout
    Sliq.append(Sout)
  return tf.stack(Sliq, axis=1)

# LSM with STDP in Liquid-Liquid (LL) connections
def run_LSM_LL_STDP(Vm, SI, Sin, SinI, spT, eta, eta1, a1, a2, in_W, W, th, n_iter, batch_s, Nin, Nrev, Ap, An, a3=0.1, eta2=0.1):
  Sliq = []
  for i in range(n_iter):
    Vm = (1-eta)*Vm + a1*(tf.matmul(SI, W) + tf.matmul(SinI, in_W))
    Sout = tf.cast(Vm>=th, dtype=tf.float32)
    Vm = Vm*tf.cast(Vm<th, dtype=tf.float32)
    SinI = (1-eta1)*SinI + a2*Sin[:,i,:]
    SI = (1-eta1)*SI + a2*Sout
    spT = (1-eta2)*spT + a3*Sout

    dwp = Ap*tf.matmul(tf.transpose(spT), Sout)/batch_s
    dwn = -An*tf.matmul(spT, tf.transpose(Sout))/batch_s
    W = W + dwp + dwn
    
    tau_astro = 10
    eta_astro = 1/tau_astro
    w_astro = 0.01
    An = An*(1-eta_astro) + (w_astro/tau_astro)*(tf.reduce_mean(tf.reduce_sum(Sout, axis=1)) - tf.reduce_mean(tf.reduce_sum(Sin[:,i,:], axis=1))) + Ap/tau_astro

    Sliq.append(Sout)
  return tf.stack(Sliq, axis=1), W, An

# LSM with STDP in Input-Liquid (IL) connections
def run_LSM_IL_STDP(Vm, SI, Sin, SinI, spT, spT_in, eta, eta1, a1, a2, in_W, W, th, n_iter, batch_s, Nin, Nrev, Ap, An, a3=0.1, eta2=0.1):
  Sliq = []
  for i in range(n_iter):
    Vm = (1-eta)*Vm + a1*(tf.matmul(SI, W) + tf.matmul(SinI, in_W))
    Sout = tf.cast(Vm>=th, dtype=tf.float32)
    Vm = Vm*tf.cast(Vm<th, dtype=tf.float32)
    SinI = (1-eta1)*SinI + a2*Sin[:,i,:]
    SI = (1-eta1)*SI + a2*Sout
    spT = (1-eta2)*spT + a3*Sout
    spT_in = (1-eta2)*spT_in + a3*Sin[:,i,:]

    dwp = Ap*tf.matmul(tf.transpose(spT_in), Sout)/batch_s
    dwn = -An*tf.matmul(tf.transpose(Sin[:,i,:]), spT)/batch_s
    in_W = in_W + dwp + dwn

    tau_astro = 10
    eta_astro = 1/tau_astro
    w_astro = 0.01
    An = An*(1-eta_astro) + (w_astro/tau_astro)*(tf.reduce_mean(tf.reduce_sum(Sout, axis=1)) - tf.reduce_mean(tf.reduce_sum(Sin[:,i,:], axis=1))) + Ap/tau_astro

    Sliq.append(Sout)
  return tf.stack(Sliq, axis=1), in_W, An

# LSM with STDP in all (LL and IL) connections
def run_LSM_full_STDP(Vm, SI, Sin, SinI, spT, spT_in, eta, eta1, a1, a2, in_W, W, th, n_iter, batch_s, Nin, Nrev, Ap, An, a3=0.1, eta2=0.1):
  Sliq = []
  for i in range(n_iter):
    Vm = (1-eta)*Vm + a1*(tf.matmul(SI, W) + tf.matmul(SinI, in_W))
    Sout = tf.cast(Vm>=th, dtype=tf.float32)
    Vm = Vm*tf.cast(Vm<th, dtype=tf.float32)
    SinI = (1-eta1)*SinI + a2*Sin[:,i,:]
    SI = (1-eta1)*SI + a2*Sout
    spT = (1-eta2)*spT + a3*Sout
    spT_in = (1-eta2)*spT_in + a3*Sin[:,i,:]

    '''
    spT_reshape = tf.reshape(spT, [batch_s, -1, 1])
    spT_rep = tf.tile(spT_reshape, [1, 1, Nrev]) # this is actually (SIrep)T from derivation
    spT_rep_in = tf.tile(spT_reshape, [1, 1, Nin]) # this is required for IL weight tuning

    spT_in_reshape = tf.reshape(spT_in, [batch_s, -1, 1])
    spT_in_rep = tf.tile(spT_in_reshape, [1, 1, Nrev]) # this is actually (SinIrep)T from derivation

    Sout_reshape = tf.reshape(Sout, [batch_s, -1, 1])
    Sout_rep = tf.tile(Sout_reshape, [1, 1, Nrev]) # this is actually (Srep)T from derivation
    Sout_rep_in = tf.tile(Sout_reshape, [1, 1, Nin]) # this is required for IL weight tuning

    Sin_reshape = tf.reshape(Sin[:,i,:], [batch_s, -1, 1])
    Sin_rep = tf.tile(Sin_reshape, [1, 1, Nrev]) # this is actually (Sinrep)T from derivation

    dwp = Ap*tf.reduce_mean(spT_in_rep*tf.transpose(Sout_rep_in,[0,2,1]), axis=0)
    dwn = -An*tf.reduce_mean(tf.transpose(spT_rep_in,[0,2,1])*Sin_rep, axis=0)
    in_W = in_W + dwp + dwn

    dwp = Ap*tf.reduce_mean(spT_rep*tf.transpose(Sout_rep), axis=0)
    dwn = -An*tf.reduce_mean(tf.transpose(spT_rep,[0,2,1])*Sout_rep, axis=0)
    W = W + dwp + dwn
    '''

    dwp = Ap*tf.matmul(tf.transpose(spT_in), Sout)/batch_s
    dwn = -An*tf.matmul(tf.transpose(Sin[:,i,:]), spT)/batch_s
    in_W = in_W + dwp + dwn
    
    dwp = Ap*tf.matmul(tf.transpose(spT), Sout)/batch_s
    dwn = -An*tf.matmul(tf.transpose(Sout), spT)/batch_s
    W = W + dwp + dwn
    
    tau_astro = 10
    eta_astro = 1/tau_astro
    w_astro = 0.01
    An = An*(1-eta_astro) + (w_astro/tau_astro)*(tf.reduce_mean(tf.reduce_sum(Sout, axis=1)) - tf.reduce_mean(tf.reduce_sum(Sin[:,i,:], axis=1))) + Ap/tau_astro

    Sliq.append(Sout)
  return tf.stack(Sliq, axis=1), in_W, W, An


Run LSM with STDP to tune weights - Only updating W_in here

In [8]:
batch_size_STDP = 50
num_iter_STDP = 20
W_lsm_tf = tf.constant(W_lsm, dtype=tf.float32)

eta = 0.1
eta1 = 0.1
a1 = 0.1
a2 = 0.1
th = 5

num_batches = data_size//batch_size_STDP
num_batches_test = data_size_test//batch_size_STDP

w_scale = 0.95
Ap = 0.15
An = 0.15

for b in range(num_batches):
  W_in_tf = tf.constant(W_in, dtype=tf.float32)
  #W_lsm_tf = tf.constant(W_lsm, dtype=tf.float32)
  if(b%50==0):
    print("completed: ", b/num_batches)
    print("An: ", An)
  Vm_tf = tf.constant(np.zeros((batch_size_STDP, N)), dtype=tf.float32)
  SI_tf = tf.constant(np.zeros((batch_size_STDP, N)), dtype=tf.float32)
  SinI_tf = tf.constant(np.zeros((batch_size_STDP, in_size)), dtype=tf.float32)
  spT_tf = tf.constant(np.zeros((batch_size_STDP, N)), dtype=tf.float32)
  spT_in_tf = tf.constant(np.zeros((batch_size_STDP, in_size)), dtype=tf.float32)

  batch_data = X_train[b*batch_size_STDP:(b+1)*batch_size_STDP]*(max_in_spikes/num_iter_STDP)
  batch_reshp = np.reshape(batch_data, (batch_size_STDP, -1, 1))
  batch_data_tiled = np.tile(batch_reshp, (1, 1, num_iter_STDP))
  batch_data_trans = np.transpose(batch_data_tiled, (0,2,1))
  batch_in = np.random.uniform(size=(batch_size_STDP , num_iter_STDP, in_size))
  batch_in[batch_in<batch_data_trans] = 1
  batch_in[batch_in<1] = 0

  batch_in_full = batch_in
  batch_in_full_tf = tf.constant(batch_in_full[:,:num_iter_STDP,:], dtype=tf.float32)

  S_liq, W_in, W_lsm, An = run_LSM_full_STDP(Vm_tf, SI_tf, batch_in_full_tf, SinI_tf, spT_tf, spT_in_tf, eta, eta1, a1, a2, W_in_tf, W_lsm_tf, th, num_iter_STDP, batch_size_STDP, in_size, N, Ap, An)
  #S_liq, W_in, An = run_LSM_IL_STDP(Vm_tf, SI_tf, batch_in_full_tf, SinI_tf, spT_tf, spT_in_tf, eta, eta1, a1, a2, W_in_tf, W_lsm_tf, th, num_iter_STDP, batch_size_STDP, in_size, N, Ap, An)


completed:  0.0
An:  0.15
completed:  0.05
An:  tf.Tensor(0.5594615, shape=(), dtype=float32)
completed:  0.1
An:  tf.Tensor(0.6881141, shape=(), dtype=float32)
completed:  0.15
An:  tf.Tensor(0.5264671, shape=(), dtype=float32)
completed:  0.2
An:  tf.Tensor(0.53462094, shape=(), dtype=float32)
completed:  0.25
An:  tf.Tensor(0.59812987, shape=(), dtype=float32)
completed:  0.3
An:  tf.Tensor(0.5895335, shape=(), dtype=float32)
completed:  0.35
An:  tf.Tensor(0.5442413, shape=(), dtype=float32)
completed:  0.4
An:  tf.Tensor(0.5689179, shape=(), dtype=float32)
completed:  0.45
An:  tf.Tensor(0.5320575, shape=(), dtype=float32)
completed:  0.5
An:  tf.Tensor(0.53252643, shape=(), dtype=float32)
completed:  0.55
An:  tf.Tensor(0.54225767, shape=(), dtype=float32)
completed:  0.6
An:  tf.Tensor(0.59749734, shape=(), dtype=float32)
completed:  0.65
An:  tf.Tensor(0.550735, shape=(), dtype=float32)
completed:  0.7
An:  tf.Tensor(0.53975606, shape=(), dtype=float32)
completed:  0.75
An:  tf

In [9]:
print("max value in W_in : ", np.max(W_in))
print("min value in W_in : ", np.min(W_in))

print("max value in W_lsm : ", np.max(W_lsm))
print("min value in W_lsm : ", np.min(W_lsm))

max value in W_in :  8.191046
min value in W_in :  -6.2364354
max value in W_lsm :  3.0031111
min value in W_lsm :  -3.4702284


Run LSM without STDP to generate liquid activations of train and test sets

In [10]:
batch_size = 200

W_lsm_tf = tf.constant(W_lsm, dtype=tf.float32)
W_in_tf = tf.constant(W_in, dtype=tf.float32)

eta = 0.1
eta1 = 0.1
a1 = 0.1
a2 = 0.1
th = 5

num_batches = data_size//batch_size
num_batches_test = data_size_test//batch_size

LSM_out_train = np.zeros((X_train.shape[0],N))
LSM_out_test = np.zeros((X_test.shape[0],N))

w_scale = 0.95

for b in range(num_batches):
  if(b%50==0):
    print("completed: ", b/num_batches)
  Vm_tf = tf.constant(np.zeros((batch_size, N)), dtype=tf.float32)
  SI_tf = tf.constant(np.zeros((batch_size, N)), dtype=tf.float32)
  SinI_tf = tf.constant(np.zeros((batch_size, in_size)), dtype=tf.float32)
  spT_tf = tf.constant(np.zeros((batch_size, N)), dtype=tf.float32)

  batch_data = X_train[b*batch_size:(b+1)*batch_size]*(max_in_spikes/num_iter)
  batch_reshp = np.reshape(batch_data, (batch_size, -1, 1))
  batch_data_tiled = np.tile(batch_reshp, (1, 1, num_iter))
  batch_data_trans = np.transpose(batch_data_tiled, (0,2,1))
  batch_in = np.random.uniform(size=(batch_size , num_iter, in_size))
  batch_in[batch_in<batch_data_trans] = 1
  batch_in[batch_in<1] = 0

  batch_in_full = batch_in
  batch_in_full_tf = tf.constant(batch_in_full, dtype=tf.float32)

  S_liq = run_LSM(Vm_tf, SI_tf, batch_in_full_tf, SinI_tf, eta, eta1, a1, a2, W_in_tf, W_lsm_tf, w_scale, th, num_iter)
  #S_liq, W_lsm = run_LSM_STDP(Vm_tf, SI_tf, test_in_tf, spT_tf, eta, eta1, a1, a2, W_lsm_tf, th, num_iter, batch_size, N)
  
  S_liq_c = tf.reduce_mean(S_liq, axis=1)
  LSM_out_train[b*batch_size:(b+1)*batch_size] = S_liq_c.numpy()

for b in range(num_batches_test):
  Vm_tf = tf.constant(np.zeros((batch_size, N)), dtype=tf.float32)
  SI_tf = tf.constant(np.zeros((batch_size, N)), dtype=tf.float32)
  SinI_tf = tf.constant(np.zeros((batch_size, in_size)), dtype=tf.float32)
  spT_tf = tf.constant(np.zeros((batch_size, N)), dtype=tf.float32)

  batch_data = X_test[b*batch_size:(b+1)*batch_size]*(max_in_spikes/num_iter)
  batch_reshp = np.reshape(batch_data, (batch_size, -1, 1))
  batch_data_tiled = np.tile(batch_reshp, (1, 1, num_iter))
  batch_data_trans = np.transpose(batch_data_tiled, (0,2,1))
  batch_in = np.random.uniform(size=(batch_size , num_iter, in_size))
  batch_in[batch_in<batch_data_trans] = 1
  batch_in[batch_in<1] = 0

  batch_in_full = batch_in
  batch_in_full_tf = tf.constant(batch_in_full, dtype=tf.float32)

  S_liq = run_LSM(Vm_tf, SI_tf, batch_in_full_tf, SinI_tf, eta, eta1, a1, a2, W_in_tf, W_lsm_tf, w_scale, th, num_iter)
  #S_liq, W_lsm = run_LSM_STDP(Vm_tf, SI_tf, test_in_tf, spT_tf, eta, eta1, a1, a2, W_lsm_tf, th, num_iter, batch_size, N)
  
  S_liq_c = tf.reduce_mean(S_liq, axis=1)
  LSM_out_test[b*batch_size:(b+1)*batch_size] = S_liq_c.numpy()

completed:  0.0
completed:  0.2
completed:  0.4
completed:  0.6
completed:  0.8


In [11]:
print("mean LSM spiking (train) : ", np.mean(LSM_out_train))
print("mean LSM spiking (test) : ", np.mean(LSM_out_test))

mean LSM spiking (train) :  0.08791386009910472
mean LSM spiking (test) :  0.08813886288187073


In [12]:
from sklearn import linear_model

print("training linear model on LSM")
clf = linear_model.SGDClassifier(max_iter=10000, tol=1e-6)
clf.fit(LSM_out_train, y_train_data)

score = clf.score(LSM_out_test, y_test_data)
print("test score = " + str(score))

print("training linear model without LSM")
clf2 = linear_model.SGDClassifier(max_iter=10000, tol=1e-6)
clf2.fit(X_train, y_train_data)

score = clf2.score(X_test, y_test_data)
print("test score without LSM = " + str(score))


training linear model on LSM
test score = 0.9664
training linear model without LSM
test score without LSM = 0.919
